In [ ]:
#Importar Arquivos e Bibliotecas
import pandas as pd
import pathlib
import yagmail

emails_df = pd.read_excel(r'Bases de Dados\Emails.xlsx')
vendas_df = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
lojas_df = pd.read_csv(r'Bases de Dados\Lojas.csv',sep=';',encoding='latin1')

for i in emails_df['E-mail']:
    emails_df['E-mail'] = emails_df['E-mail'].replace(i,f'danielgatuu+{i[21:-10]}@gmail.com')

vendas_df = vendas_df.merge(lojas_df, on='ID Loja')
vendas_df = vendas_df.merge(emails_df, on='Loja')

display(vendas_df)

#Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador
dicionario_lojas = {}
for loja in lojas_df['Loja']:
    dicionario_lojas[loja] = vendas_df.loc[vendas_df['Loja']==loja, :]
    
dia_indicador = vendas_df['Data'].max()
print(dia_indicador)

#Salvar a planilha na pasta de backup
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')
arquivo_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivo_pasta_backup]

# Criar as pastas
for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
        
    # Salvar dentro da pasta
    nome_arquivo = f'{dia_indicador.day}-{dia_indicador.month}-{dia_indicador.year}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)

#Definição de metas
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

#Calcular o indicador para cada loja
for loja in dicionario_lojas:    
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador, :]

    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #diversidade de produtos
    qtde_vendas_ano = len(vendas_loja['Produto'].unique())
    qtde_vendas_dia = len(vendas_loja_dia['Produto'].unique())

    #ticket médio ano
    valor_vendas_ano = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_vendas_ano['Valor Final'].mean()
    
    #ticket médio ano
    valor_vendas_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_vendas_dia['Valor Final'].mean()
    
    #Enviar por e-mail para os gerentes
    anexos = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.day}-{dia_indicador.month}-{dia_indicador.year}_{loja}.xlsx'

    usuario = yagmail.SMTP(user='SeuEmail@gmail.com', password='xxxxxxxx')
    nome = emails_df.loc[emails_df['Loja']==loja,'Gerente'].values[0]

    def cor_simbolo(valor,meta):
        if valor >= meta:
            return 'green'
        else:
            return 'red'

    cor_faturamento_dia = cor_simbolo(faturamento_dia,meta_faturamento_dia)
    cor_diversidade_produtos_dia = cor_simbolo(qtde_vendas_dia,meta_qtdeprodutos_dia)
    cor_ticket_dia = cor_simbolo(ticket_medio_dia,meta_ticketmedio_dia)

    cor_faturamento_ano = cor_simbolo(faturamento_ano,meta_faturamento_ano)
    cor_diversidade_produtos_ano = cor_simbolo(qtde_vendas_ano,meta_qtdeprodutos_ano)
    cor_ticket_ano = cor_simbolo(ticket_medio_ano,meta_ticketmedio_ano)

    corpo_email = f''' 
    <p>Bom dia, {nome}</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <html>
    <body>
    <table style="width:60%"> 
      <tr>
        <th><font size="+0.5">Indicador</font></th>
        <th><font size="+0.5">Valor Dia</font></th>
        <th><font size="+0.5">Meta Dia</font></th>
        <th><font size="+0.5">Cenário Dia</font></th>
      </tr>
      <tr>
        <td style="text-align: center">Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font size="+4" , color="{cor_faturamento_dia}">☜</font></td>
      </tr>
      <tr>
        <td style="text-align: center">Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_vendas_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font size="+4" , color="{cor_diversidade_produtos_dia}">☜</font></td>
      </tr>
      <tr>
        <td style="text-align: center">Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font size="+4" , color="{cor_ticket_dia}">☜</font></td>
      </tr>
    </table>

    </body>
    </html>

    <html>

    <body>

    <table style="width:60%">
      <tr>
        <th><font size="+0.5">Indicador</font></th>
        <th><font size="+0.5">Valor Ano</font></th>
        <th><font size="+0.5">Meta Ano</font></th>
        <th><font size="+0.5">Cenário Ano</font></th>
      </tr>
      <tr>
        <td style="text-align: center">Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:,.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:,.2f}</td>
        <td style="text-align: center"><font size="+4" , color="{cor_faturamento_ano}">☜</font></td>
      </tr>
      <tr>
        <td style="text-align: center">Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_vendas_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font size="+4" , color="{cor_diversidade_produtos_ano}">☜</font></td>
      </tr>
      <tr>
        <td style="text-align: center">Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font size="+4" , color="{cor_ticket_ano}">☜</font></td>
      </tr>
    </table>

    </body>
    </html>

    '''
    corpo_email = corpo_email.replace('\n', ' ')

    usuario.send(to=emails_df.loc[emails_df['Loja']==loja,'E-mail'].values[0],
                 subject= f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}',
                 contents= corpo_email,
                 attachments=anexos)
    print(f'E-mail da Loja {loja} Enviado')

# Criar ranking para diretoria
faturamento_loja = vendas_df.groupby('Loja')[['Loja','Valor Final']].sum()
faturamento_loja_ano = faturamento_loja.sort_values(by = 'Valor Final',ascending = False)
display(faturamento_loja_ano)

nome_arquivo = f'{dia_indicador.day}-{dia_indicador.month}-{dia_indicador.year}_Ranking Anual.xlsx'
faturamento_loja_ano.to_excel(fr'Backup Arquivos Lojas\ {nome_arquivo}')

vendas_dia = vendas_df.loc[vendas_df['Data'] == dia_indicador,:]
faturamento_loja_dia = vendas_dia.groupby('Loja')[['Loja','Valor Final']].sum()
faturamento_loja_dia = faturamento_loja_dia.sort_values(by = 'Valor Final',ascending = False)
display(faturamento_loja_dia)

nome_arquivo = f'{dia_indicador.day}-{dia_indicador.month}-{dia_indicador.year}_Ranking Do Dia.xlsx'
faturamento_loja_dia.to_excel(fr'Backup Arquivos Lojas\ {nome_arquivo}')

#Enviar e-mail para diretoria
corpo_email = f'''
<p><font size="+0.5">Prezados, bom dia.</font></p>

<p><font size="+0.5">Tabela Baseada Apenas No Faturamento.</font></p>

<html>
<body>
<table style="width:60%"> 
  <tr>
    <th><font size="+0.5">Melhor Loja do Dia</font></th>
    <th><font size="+0.5">Pior Loja do Dia</font></th>
    <th><font size="+0.5">Melhor Loja do Ano</font></th>
    <th><font size="+0.5">Pior Loja do Ano</font></th>
  </tr>
  
  <tr>
    <td style="text-align: center">{faturamento_loja_dia.index[0]}</td>
    <td style="text-align: center">{faturamento_loja_dia.index[-1]}</td>
    <td style="text-align: center">{faturamento_loja_ano.index[0]}</td>
    <td style="text-align: center">{faturamento_loja_ano.index[-1]}</td>
  </tr>  
  
   <tr>
    <th><font size="+0.5">Faturamento</font></th>
    <th><font size="+0.5">Faturamento</font></th>
    <th><font size="+0.5">Faturamento</font></th>
    <th><font size="+0.5">Faturamento</font></th>
  </tr>
  
   <tr>
    <td style="text-align: center">R${faturamento_loja_dia.iloc[0, 0]:.2f}</td>
    <td style="text-align: center">R${faturamento_loja_dia.iloc[-1, 0]:.2f}</td>
    <td style="text-align: center">R${faturamento_loja_ano.iloc[0, 0]:,.2f}</td>
    <td style="text-align: center">R${faturamento_loja_ano.iloc[-1, 0]:,.2f}</td>
  </tr> 
    
</table>

</body>
</html>

<p><font size="+0.5">Segue em anexo os rankings do ano e do dia de todas as lojas.</font></p>

<p><font size="+0.5">Qualquer dúvida estou à disposição.</font></p>

<p><font size="+0.5">Att.Daniel</font></p>

'''
corpo_email = corpo_email.replace('\n', ' ')

anexo1 = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.day}-{dia_indicador.month}-{dia_indicador.year}_Ranking Do Dia.xlsx'
anexo2 = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.day}-{dia_indicador.month}-{dia_indicador.year}_Ranking Anual.xlsx'

usuario = yagmail.SMTP(user='SeuEmail@gmail.com', password='xxxxxxxx')
usuario.send(to=emails_df.loc[emails_df['Loja']=='Diretoria','E-mail'].values[0],
             subject= f'Ranking Do Dia {dia_indicador.day}/{dia_indicador.month}',
             contents= corpo_email,
             attachments= [anexo1,anexo2])
print(f'E-mail da Diretoria Enviado')